In [1]:
print("all ok")

all ok


In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
model = ChatGroq(model="qwen/qwen3-32b")
model.invoke("what is the capital of France?").content

"<think>\nOkay, the user is asking for the capital of France. Let me think. I remember from school that France's capital is Paris. But wait, maybe I should verify that. Let me see... Yes, Paris is definitely the capital. It's a major city in Europe, known for landmarks like the Eiffel Tower and the Louvre. I don't think there's any other city that could be confused with it in France. The government is based there, and it's the largest city in the country. I'm pretty confident the answer is Paris. I should just state that clearly.\n</think>\n\nThe capital of France is **Paris**. It is renowned for its cultural, historical, and political significance, as well as iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral."

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings_model.embed_query("what is the capital of France?")

[0.04049589857459068,
 -0.04480205848813057,
 -0.03519536182284355,
 -0.039978861808776855,
 0.060929879546165466,
 -0.0020229103974997997,
 0.0018945873016491532,
 -0.02098630554974079,
 0.00390483345836401,
 0.07453471422195435,
 -0.018631644546985626,
 0.007455417420715094,
 -0.005498720332980156,
 0.0277552530169487,
 0.02441469207406044,
 -0.04078073427081108,
 0.006169943604618311,
 0.028121715411543846,
 0.047293782234191895,
 -0.011507853865623474,
 0.01713664084672928,
 0.008143678307533264,
 -0.03534378856420517,
 0.024886861443519592,
 0.018857652321457863,
 -0.05387217551469803,
 0.012068687006831169,
 -0.04790126904845238,
 -0.012439027428627014,
 -0.0018709597643464804,
 -0.07322046160697937,
 0.04119649529457092,
 0.003926463890820742,
 -0.0114192645996809,
 0.027035439386963844,
 -0.05288225784897804,
 -0.004346303641796112,
 0.023509696125984192,
 -0.00741075212135911,
 0.0356050580739975,
 -0.00813733134418726,
 -0.009293914772570133,
 -0.04593217745423317,
 0.0109800

1. Data Ingestion

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [7]:
import os
file_path = os.path.join(os.getcwd(),"data","sample.pdf")

In [9]:
loader = PyPDFLoader(file_path)
documents = loader.load()

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=150,length_function=len)
docs = text_splitter.split_documents(documents)

In [12]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs,embeddings_model)

In [28]:
relevant_doc=vectorstore.similarity_search("llama2 fine tunring benchmark expriements", k =10)
relevant_doc

[Document(id='5ac4ef7d-2acb-47d9-a3d2-f1329f5de17c', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/arunkota/Documents/MLOPS/document_portal/notebook/data/sample.pdf', 'total_pages': 77, 'page': 73, 'page_label': '74'}, page_content='Llama 2\n7B 0.28 0.25 0.29 0.50 0.36 0.37 0.21 0.34 0.32 0.50 0.28 0.19 0.26 0.32 0.44 0.51 0.30 0.2513B 0.24 0.25 0.35 0.50 0.41 0.36 0.24 0.39 0.35 0.48 0.31 0.18 0.27 0.34 0.46 0.66 0.35 0.2834B 0.27 0.24 0.33 0.56 0.41 0.36 0.26 0.32 0.36 0.53 0.33 0.07 0.26 0.30 0.45 0.56 0.26 0.3570B 0.31 0.29 0.35 0.51 0.41 0.45 0.27 0.34 0.40 0.52 0.36 0.12 0.28 0.31 0.45 0.65 0.33 0.20\nFine-tuned'),
 Document(id='d321531e-9cda-4c8d-81b4-9e846a833fda',

In [15]:
retriever = vectorstore.as_retriever()

In [ ]:
prompt_template=""" 
    Answer the question based on the context provided below.
    If the context does not contain sufficient information, respond with :
    "I dont have enough information about this."
    Context :{context}
    Question : {question}
    Answer :
 """

In [23]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template = prompt_template,
    input_variables=["context", "question"]
    
)

In [24]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [25]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [26]:
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
    
)

In [29]:
rag_chain.invoke("tll me about the llama2 finetuning benchmark experiements")

'<think>\nOkay, let\'s tackle this question about the Llama 2 fine-tuning benchmark experiments. First, I need to look at the context provided to see what information is available.\n\nThe context has several tables with numbers, but it\'s not very clearly formatted. Let me parse through it step by step. The first part mentions different Llama 2 models with varying parameters (7B, 13B, 34B, 70B) followed by a series of numbers. Then there\'s a section labeled "Fine-tuned" with a 65B model and a bunch of numbers. After that, there\'s another section with Llama 2 models again and more numbers, followed by "Fine-tuned" and some other models like ChatGPT, MPT-instruct, Falcon-instruct, each with their own numbers. Then a table labeled "Table 3" with performance on academic benchmarks.\n\nThe user is asking specifically about the Llama 2 fine-tuning benchmark experiments. The context does mention "Fine-tuned" sections for Llama 2 models. For example, under the "Fine-tuned" part after the ini

Take 10 PDF keep in the same directory and create RAG on top of it
